In [1]:
import os
os.chdir("../")

In [2]:
from project import logger
from project.utils.common import clean_data,read_yaml_file,read_data_from_s3

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

import mlflow
import boto3
import pandas as pd


In [3]:
config=read_yaml_file("config/config.yaml")
s3=boto3.client("s3")

In [4]:
config.mlflow.experiment_name=config.mlflow.experiment_name.split("_")[0]+"_"+str(int(config.mlflow.experiment_name.split("_")[1])+1)
config.to_yaml("config/config.yaml")
config=read_yaml_file("config/config.yaml")

bucket=config.s3.bucket
data_object=config.s3.save_object
tracking_uri=config.mlflow.tracking_uri
experiment_name=config.mlflow.experiment_name
model_name=config.mlflow.model_name

In [5]:
mlflow.set_tracking_uri(tracking_uri)

In [6]:
mlflow.set_experiment(experiment_name)

2024/07/03 11:34:56 INFO mlflow.tracking.fluent: Experiment with name 'Experiment_7' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://sentimet-analysis/artifacts/7', creation_time=1719986698853, experiment_id='7', last_update_time=1719986698853, lifecycle_stage='active', name='Experiment_7', tags={}>

In [7]:
data=read_data_from_s3(bucket,data_object)

In [8]:
data

,Sentence,Sentiment
0,geosolut technolog leverag benefon 's gp solut...,positive
1,esi low 1.50 2.50 bk real possibl,negative
2,last quarter 2010 componenta 's net sale doubl...,positive
3,accord finnish-russian chamber commerc major c...,neutral
4,swedish buyout firm sold remain 22.4 percent s...,neutral
...,...,...
5831,rise cost forc packag produc huhtamaki axe 90 ...,negative
5832,nordic walk first use summer train method cros...,neutral
5833,accord ship compani vike line eu decis signifi...,neutral
5834,build home improv trade sale decreas 22.5 eur ...,neutral


In [9]:
vectorizer=TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['Sentiment'])

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [10]:
X_train,X_test,y_train,y_test=train_test_split(tfidf_df,data['Sentiment'],test_size=0.3)

In [11]:
nb=MultinomialNB()
svc=SVC()
models={'naive_baise':nb,'Support_Vecor_classifies':svc}

In [12]:
def train_and_log_model(model, model_name):
    with mlflow.start_run(run_name=model_name):
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')

        # Log metrics
        mlflow.log_param("model_name",model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)

        # Log the model
        mlflow.sklearn.log_model(model, "model")
        
        

In [17]:
def train():
    logger.info(">>>>>>>>>>Experiment started>>>>>>>>>>")
    for model in models:
        try:
            train_and_log_model(models[model], model)
        except Exception as e:
            logger.error("Something went wrong {}".format(e))
        
    logger.info(">>>>>>>>>>>Experiment completed")

    

In [14]:
def register_model():
    client = mlflow.tracking.MlflowClient()
    try:
        # Get the experiment
        experiment = client.get_experiment_by_name(experiment_name)
        if experiment is None:
            raise ValueError(f"Experiment {experiment_name} not found.")
        
        # Get all runs from the experiment
        runs = client.search_runs(experiment_ids=[experiment.experiment_id])
        if not runs:
            raise ValueError(f"No runs found for experiment {experiment_name}.")

        # Find the best run based on accuracy
        best_run = max(runs, key=lambda run: run.data.metrics.get('accuracy', 0))
        best_run_id = best_run.info.run_id
        latest_accuracy = best_run.data.metrics.get('accuracy', 0)

        # Load the best model
        best_model = mlflow.sklearn.load_model(f"runs:/{best_run_id}/model")

        try:
            # Check if there are any previously registered versions of the model
            latest_versions = client.get_latest_versions(model_name, stages=["None"])
        except mlflow.exceptions.RestException as e:
            if e.error_code == 'RESOURCE_DOES_NOT_EXIST':
                latest_versions = []
            else:
                raise e
        
        if not latest_versions:
            # No previous versions, register the new model
            model_uri = f"runs:/{best_run_id}/model"
            mlflow.register_model(model_uri, model_name)
            logger.info("Model has been registered.")
            
        else:
            # Check the accuracy of the latest registered model version
            latest_version = latest_versions[-1]
            run_id = latest_version.run_id
            run = client.get_run(run_id)
            last_accuracy = run.data.metrics.get("accuracy", 0)

            # Compare accuracies and register the new model if it's better
            if latest_accuracy > last_accuracy:
                model_uri = f"runs:/{best_run_id}/model"
                mlflow.register_model(model_uri, model_name)
                logger.info("Latest version of the model has been registered.")
                
                
            else:
                logger.info("Last version accuracy is greater than or equal to the latest version.")

        # Save the updated configuration
        
    except mlflow.exceptions.RestException as e:
        logger.error(f"Error during model registration: {e}")
        raise e
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        raise e


In [15]:
train()

In [16]:
register_model()